### Prototype Code : Prompt Generator

In [ ]:
# Prompt Generator + LLM

# CONTEXT
template = """
You are a quality assurance expert that generates functional test cases for websites. You take in a UI element and you generate a functional test case.

Here is the UI element (some elements have a link attached to them): {question}
ONLY output in the following format: 
"Objective"~"Preconditions"~"Test Steps"~"Expected Result"

DO NOT output any other text. DO NOT output 'Here are the test cases...', your output should be like the example output below.

Example Input:
Link Element: Home with URL : https://bicol-u.edu.ph/
Link Element: Academics with URL : https://bicol-u.edu.ph/#
...

Example Output:
"Verify the functionality of the Link Element 'Home'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Home\'' '3. Verify if the webpage opens in a new tab/window.'"~"Webpage 'https://bicol-u.edu.ph/' should open in a new tab/window."
"Verify the functionality of the Link Element 'Academics'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Academics\'' '3. Verify if the link url changes to \'https://bicol-u.edu.ph/#\'' '4. Verify if a dropdown below \'Academics\' is visible'"~"A dropdown should show below 'Academics', but the webpage does not change"
...

"""

# Load Model Chain
def load_model_chain(template : str =  template, model_str : str = "llama3.1"):
    # Prompt
    prompt = ChatPromptTemplate.from_template(template)
    # Model
    model = OllamaLLM(model=model_str)
    # Chain
    chain = prompt | model
    return chain

# Create Test Case Data
def create_test_cases(data, chain, model_str : str = "llama3.1" , template : str = template, batch_size : int = 10):
    
    return_data = []
    
    for sub_data in data:
        element_test_cases = []
        i = 0
        j = 0
        print(f"Batch Number: {ceil(len(sub_data)/batch_size)}")
        while (j<ceil(len(sub_data)/batch_size)):
            print(f"[{j}] Batch {str(len(sub_data[i:i+batch_size]))}")
            appending = []
            for dat in sub_data[i:i+batch_size]:
                appending.append(chain.invoke({"question": str(dat)}))
            element_test_cases.append(appending)
            i+=batch_size
            j+=1
            prompt = ChatPromptTemplate.from_template(template)
            model = OllamaLLM(model=model_str)
            chain = prompt | model
        return_data.append(element_test_cases)
    return return_data


### Version 1 : Dissection and Simplification

In [ ]:
# Langchain and Ollama
import langchain
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

# CONTEXT V1
template = """
You are a quality assurance expert that generates functional test cases for websites. You take in a UI element and you generate a functional test case for usability.

Some UI elements have a link attached to them and other properties.
ONLY output in the following format: 
"Objective"~"Preconditions"~"Test Steps"~"Expected Result"
DO NOT output any other text. DO NOT output 'Here are the test cases...', your output should be like the example output below.

Example Input:
Link Element: Home with URL : https://bicol-u.edu.ph/
Link Element: Academics with URL : https://bicol-u.edu.ph/#
...

Example Output:
"Verify the functionality of the Link Element 'Home'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Home\'' '3. Verify if the webpage opens in a new tab/window.'"~"Webpage 'https://bicol-u.edu.ph/' should open in a new tab/window."
"Verify the functionality of the Link Element 'Academics'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Academics\'' '3. Verify if the link url changes to \'https://bicol-u.edu.ph/#\'' '4. Verify if a dropdown below \'Academics\' is visible'"~"A dropdown should show below 'Academics', but the webpage does not change"
...

Here is the UI element : {question}
"""

# Prompt Generator + LLM
# TODO : Experiment With Prompts
# TODO : Create Altered Version For Fine-Tuned Model Version (aka System Proper)

# Load Model Chain
def load_model_chain(template : str =  template, model_str : str = "llama3.1"):
    prompt = ChatPromptTemplate.from_template(template)
    model = OllamaLLM(model=model_str)
    chain = prompt | model
    return chain

# Create Test Case Data
def create_test_cases(data, model_str : str = "llama3.1" , template : str = template):
    
    # Load LLM Chain
    chain = load_model_chain(template, model_str)

    # Return Data
    return_data = []
    
    for item in data:
        return_data.append(chain.invoke({"question": str(item)}))
        # LLM Reset To Free Up Context
        chain = load_model_chain(template, model_str)

    return return_data


##### Version 1 (System Proper):

In [ ]:
# Langchain and Ollama
import langchain
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

# CONTEXT V1
template = """ Question:\n Generate test case for the following UI element: : {question} """
model_str = "llama3.1"

# Prompt Generator + LLM
# Load Model Chain
def load_model_chain(template : str =  template, model_str : str = model_str):
    # TODO : Plug In Fine Tuned Model
    prompt = ChatPromptTemplate.from_template(template)
    model = OllamaLLM(model=model_str)
    chain = prompt | model
    return chain

# Create Test Case Data
def create_test_cases(data):
    
    # Load LLM Chain
    chain = load_model_chain()

    # Return Data
    return_data = []
    
    for item in data:
        return_data.append(chain.invoke({"question": str(item)}))
        # LLM Reset To Free Up Context
        chain = load_model_chain()

    return return_data


##### Version 2: More Website Context

In [ ]:
# CONTEXT V2
# Aim to fix:
#   - url concern
#   - 

template = """
You are a quality assurance expert that generates functional test cases for websites. You take in a UI element and you generate a functional test case for usability.

Some UI elements have a link attached to them and other properties.
ONLY output in the following format: 
"Objective"~"Preconditions"~"Test Steps"~"Expected Result"
DO NOT output any other text. DO NOT output 'Here are the test cases...', your output should be like the example output below.

Example Input:
Link Element: Home with URL : https://bicol-u.edu.ph/
Link Element: Academics with URL : https://bicol-u.edu.ph/#
...

Example Output:
"Verify the functionality of the Link Element 'Home'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Home\'' '3. Verify if the webpage opens in a new tab/window.'"~"Webpage 'https://bicol-u.edu.ph/' should open in a new tab/window."
"Verify the functionality of the Link Element 'Academics'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Academics\'' '3. Verify if the link url changes to \'https://bicol-u.edu.ph/#\'' '4. Verify if a dropdown below \'Academics\' is visible'"~"A dropdown should show below 'Academics', but the webpage does not change"
...

Here is the UI element : {question}
The ui element is from the following website : {url}
"""

In [ ]:
"""
Generate a usability test case for the following UI element:

UI Element: Link
Element Name: 'All News'

Output should include:
1. Objectives for the link element
2. Preconditions for the link element
3. Test Steps for the link element
4. Expected Output for the link element

"""

In [1]:
# V2 Template
# TODO : Improve UI Element format for prompting
# TODO : Test against input texts more
# Used Techniques : Few shot prompting, specify context, and specify format.

template_2 = """
You are a quality assurance expert that generates functional test cases (usability) for websites. 

Generate a functional test case for usability for the following UI Element:
{ui_element}
The UI Element is from the following link:
{url}

Output should be in the form:
"Objective"~"Preconditions"~"Test Steps"~"Expected Output"

Example Input:
Link Element: Home with URL : https://bicol-u.edu.ph/ from url : https://bicol-u.edu.ph/
Link Element 'Battle of New Orleans' With URL https://en.wikipedia.org/wiki/Battle_of_New_Orleans from url : https://en.wikipedia.org/

Example Output:
"Verify the functionality of the Link Element 'Home'"~"The user is on the webpage 'https://bicol-u.edu.ph/'"~"'1. User navigates to the webpage \'https://bicol-u.edu.ph/\'' '2. Click on Link Element \'Home\'' '3. Verify if the webpage opens in a new tab/window.'"~"Webpage 'https://bicol-u.edu.ph/' should open in a new tab/window."
"Verify the functionality of the Link Element 'Battle of New Orleans'"~"The user is on the webpage 'https://en.wikipedia.org/'"~"'1. User navigates to the webpage 'https://en.wikipedia.org/' '' '2. Click on Link Element 'Battle of New Orleans'' '3. Verify if the link url changes to 'https://en.wikipedia.org/wiki/Battle_of_New_Orleans' '4. Verify if the webpage opens in a new tab/window'"~"Webpage 'https://en.wikipedia.org/wiki/Battle_of_New_Orleans/' should open in a new tab/window."

"""

In [ ]:
peer_reviewed_template ="""
Prompt:
*"You are an expert in software quality assurance specializing in usability testing. Given a UI element, generate a set of functional usability test cases with detailed test scenarios.

The UI element could be a button, link, text, or input field (some elements have a link attached to them). Consider the following usability aspects:

Accessibility (keyboard navigation, screen reader support)
Responsiveness (behavior across different screen sizes)
Feedback (hover effects, click responses, error messages)
Interactivity (expected behavior when clicked, typed into, or focused)
User experience (clarity of labels, ease of use)

For each test case, include:

Objective (What the test aims to verify)
Preconditions (Any setup or conditions required before testing)
Test Steps (Step-by-step actions to perform the test)
Expected Result (Expected outcome if the UI element works correctly)

Generate test cases in a structured format: "Objective"~"Preconditions"~"Test Steps"~"Expected Result" "*
DO NOT output 'Here are the test cases...'
"""

# Assuming LLM Temperature 0.1
revised_peer_reviewed_template = """
You are an expert in software quality assurance specializing in usability testing. Given a UI element, generate a set of functional usability test cases with detailed test scenarios.

The UI element could be a button, link, text, or input field (some elements have a link attached to them). Consider the attribute attached to the UI element as the attribute acts as a hint about the primary focus of the test case. Also consider the following usability aspects:

Accessibility (keyboard navigation, screen reader support, etc.)
Responsiveness (behavior across different screen sizes, etc.)
Feedback (hover effects, click responses, error messages, etc.)
Interactivity (expected behavior when clicked, typed into, or focused, etc.)
User experience (clarity of labels, ease of use, etc.)

Only output the following, separated by a ~:

Objective (What the test aims to verify)
Preconditions (Any setup or conditions required before testing)
Test Steps (Step-by-step actions to perform the test)
Expected Result (Expected outcome if the UI element works correctly)

Your output should only be in the following structured format but do not include the format in the output: "Objective"~"Preconditions"~"Test Steps"~"Expected Result"
DO NOT output anything else but one test case in the said format.

Here is the UI Element: {ui_element}
The UI Element is from the following website URL : {url}
"""

# Common error exists that can be removed through python string
common_error = "Objective~Preconditions~Test Steps~Expected Result"

#Suggestions:
# - Consider the attribute attached to the UI element as the attribute acts as a hint in what the test case should focus on.

In [5]:
# Langchain and Ollama
import langchain
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

# Dataset Generator Version TESTING
template = """
You are an expert in software quality assurance specializing in usability testing. Given a UI element, generate a set of functional usability test cases with detailed test scenarios.

The UI element could be a button, link, text, or input field (some elements have a link attached to them). Consider the attribute attached to the UI element as the attribute acts as a hint about the primary focus of the test case. Also consider the following usability aspects:

Accessibility (keyboard navigation, screen reader support, etc.)
Responsiveness (behavior across different screen sizes, etc.)
Feedback (hover effects, click responses, error messages, etc.)
Interactivity (expected behavior when clicked, typed into, or focused, etc.)
User experience (clarity of labels, ease of use, etc.)

Only output the following, separated by a ~:

Objective (What the test aims to verify)
Preconditions (Any setup or conditions required before testing)
Test Steps (Step-by-step actions to perform the test)
Expected Result (Expected outcome if the UI element works correctly)

Your output should only be in the following structured format but do not include the format in the output: "Objective"~"Preconditions"~"Test Steps"~"Expected Result"
DO NOT output anything else but one test case in the said format.

Here is the UI Element: {ui_element}
The UI Element is from the following website URL : {url}
"""

common_error = "Objective~Preconditions~Test Steps~Expected Result"

ui_element = "Clickable Button Element 'Accessibility Button'"
url = "https://www.dost.gov.ph/"

# Load Model Chain
def load_model_chain(template : str =  template, model_str : str = "llama3.1", temperature=0.1):
    prompt = ChatPromptTemplate.from_template(template)
    model = OllamaLLM(model=model_str, temperature=temperature)
    chain = prompt | model
    return chain

# Create Test Case Data
def create_test_cases(data, model_str : str = "llama3.1" , template : str = template, url : str = url):
    
    # Load LLM Chain
    chain = load_model_chain(template, model_str)

    # Return Data
    return_data = []
    
    i = 0
    total = len(data)
    for item in data:
        return_data.append(chain.invoke({"ui_element": str(item), "url": url}))
        # LLM Reset To Free Up Context
        chain = load_model_chain(template, model_str)
        print(f"test case {i} out of {total} generated")

    return return_data

chain = load_model_chain()
answer = chain.invoke({"ui_element": ui_element, "url": url})
print(answer.replace(common_error, ""))
chain = load_model_chain()


Verify that the Accessibility Button can be navigated to using keyboard navigation and is announced by a screen reader.~The user has accessed the website at https://www.dost.gov.ph/ and has enabled keyboard navigation in their browser settings. The user also has a screen reader software installed and enabled on their device.~1. Press the Tab key repeatedly until the Accessibility Button receives focus.
2. Use the Enter key to click the Accessibility Button.
3. Simultaneously press the Windows key + D (Windows) or Command + Shift + D (Mac) to open the screen reader software and have it announce the button's label.
4. Verify that the screen reader announces the button's label correctly.~The screen reader should announce the button's label, which is likely "Accessibility Button", when clicked using keyboard navigation.


In [7]:
split_test = answer.replace(common_error, "")
split_test = split_test.split("~")
for split in split_test:
    print(split)
    print("\n")




Verify that the Accessibility Button can be navigated to using keyboard navigation and is announced by a screen reader.




The user has accessed the website at https://www.dost.gov.ph/ and has enabled keyboard navigation in their browser settings. The user also has a screen reader software installed and enabled on their device.




1. Press the Tab key repeatedly until the Accessibility Button receives focus.
2. Use the Enter key to click the Accessibility Button.
3. Simultaneously press the Windows key + D (Windows) or Command + Shift + D (Mac) to open the screen reader software and have it announce the button's label.
4. Verify that the screen reader announces the button's label correctly.




The screen reader should announce the button's label, which is likely "Accessibility Button", when clicked using keyboard navigation.




In [9]:
# Further Testing

ui_element = "Link Element Without Name (might be an image) With URL https://www.dost.gov.ph/77-honor-awards-program-hap-story.html With target attribute _blank"
url = "https://www.dost.gov.ph/"

chain = load_model_chain()
answer = chain.invoke({"ui_element": ui_element, "url": url})
chain = load_model_chain()
split_test = answer.replace(common_error, "")
split_test = split_test.split("~")
for split in split_test:
    print(split)
    print("\n")


Verify that clicking on a link element without name (image) opens the target URL in a new browser tab.


The user is on the webpage with the link element, and the device has an active internet connection.


1. Navigate to the webpage containing the link element.
2. Use keyboard navigation to focus on the link element.
3. Click on the link element using the mouse or touch input.
4. Verify that a new browser tab opens with the target URL (https://www.dost.gov.ph/77-honor-awards-program-hap-story.html).
5. Check if the original webpage remains open in the background.


The new browser tab should be opened with the target URL, and the original webpage should remain active in the background.




##### Resulting Version
- Different (Tested) Prompt
- Added a failsafe substring removal in case LLM commits common error
- Added temperature parameters

In [ ]:
# Langchain and Ollama
import langchain
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

# Dataset Generator Version TESTING
template = """
You are an expert in software quality assurance specializing in usability testing. Given a UI element, generate a set of functional usability test cases with detailed test scenarios.

The UI element could be a button, link, text, or input field (some elements have a link attached to them). Consider the attribute attached to the UI element as the attribute acts as a hint about the primary focus of the test case. Also consider the following usability aspects:

Accessibility (keyboard navigation, screen reader support, etc.)
Responsiveness (behavior across different screen sizes, etc.)
Feedback (hover effects, click responses, error messages, etc.)
Interactivity (expected behavior when clicked, typed into, or focused, etc.)
User experience (clarity of labels, ease of use, etc.)

Only output the following, separated by a ~:

Objective (What the test aims to verify)
Preconditions (Any setup or conditions required before testing)
Test Steps (Step-by-step actions to perform the test)
Expected Result (Expected outcome if the UI element works correctly)

Your output should only be in the following structured format but do not include the format in the output: "Objective"~"Preconditions"~"Test Steps"~"Expected Result"
DO NOT output anything else but one test case in the said format.

Here is the UI Element: {ui_element}
The UI Element is from the following website URL : {url}
"""

common_error = "Objective~Preconditions~Test Steps~Expected Result"

# Load Model Chain
def load_model_chain(template : str =  template, model_str : str = "llama3.1", temperature=0.1):
    prompt = ChatPromptTemplate.from_template(template)
    model = OllamaLLM(model=model_str, temperature=temperature)
    chain = prompt | model
    return chain

# Create Test Case Data
def create_test_cases(data, model_str : str = "llama3.1" , template : str = template, url : str = url, common_error : str = common_error):
    
    # Load LLM Chain
    chain = load_model_chain(template, model_str)

    # Return Data
    return_data = []
    
    i = 0
    total = len(data)
    for item in data:
        return_data.append((chain.invoke({"ui_element": str(item), "url": url})).replace(common_error, ""))
        # LLM Reset To Free Up Context
        chain = load_model_chain(template, model_str)
        print(f"test case {i} out of {total} generated")

    return return_data


##### Version 3
- add Common Error Fixer / Validator Function
- add rompt Adjustments

In [ ]:
## Prompt Generator + LLM
# Langchain and Ollama
import langchain
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

TEMPLATE_SETTING = 1
DEBUG_SETTING = 1

if (TEMPLATE_SETTING == 0):
    # Template for non fine tuned model
    # CONTEXT V3
    template = """
    
    """

    model_str = "llama3.1"
elif (TEMPLATE_SETTING == 1):
    # template and model for fine-tuned version
    template = """Generate a test case for the following UI element: {ui_element} from the website: {url}"""
    model_str = ""

# OPTIONAL FUNCTION COMMON ERROR REMOVAL/ADJUSTMENT
def remove_common_error(output : str):
    # Errors are strings that commonly appear as errors in the intended output
    errors = ["Objective~Preconditions~Test Steps~Expected Result", "Objective~"]
    # Adjustments are strings that are not intended to be deleted, but instead adjusted
    adjustments = ["Preconditions~", "Test Steps~", "Expected Result~"]
    for error in errors:
        output = output.replace(error, "")
    for adjustment in adjustments:
        output = output.replace(adjustment, "~")

def load_model_chain(template : str =  template, model_str : str = model_str, temperature=0):
    prompt = ChatPromptTemplate.from_template(template)
    model = OllamaLLM(model=model_str, temperature=temperature)
    chain = prompt | model
    return chain

def create_test_cases(data, model_str : str = model_str , template : str = template, url : str = "placeholder", common_error : str = common_error):
    
    # Load LLM Chain
    chain = load_model_chain(template, model_str)

    # Return Data
    return_data = []
    
    i = 0
    total = len(data)
    for item in data:
        test_case = chain.invoke({"ui_element": str(item), "url": url})
        return_data.append(test_case)
        # LLM Reset To Free Up Context
        chain = load_model_chain(template, model_str)
        if (DEBUG_SETTING == 1):
            print(f"test case {i} out of {total} generated")
        i += 1

    return return_data